In [1]:
import cv2
import numpy as np
import os

In [2]:
import cv2
import numpy as np
import os

def detect_icons(image_path, template_paths, scales, threshold, output_folder, icon_name):
    # Load and preprocess the main image
    img = cv2.imread(image_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to smooth the image
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)

    # Apply Canny edge detection to extract sharp edges
    img_edges = cv2.Canny(img_blur, threshold1=100, threshold2=200)

    # Create a copy of the image for drawing detections
    img_detected = img.copy()
    detected_points = []  # To store detected bounding boxes
    count_detected = 0

    # Iterate over each template
    for template_path in template_paths:
        # Load and preprocess the template
        template = cv2.imread(template_path, 0)  # Load as grayscale
        template_blur = cv2.GaussianBlur(template, (5, 5), 0)
        
        # Apply Canny edge detection on the template
        template_edges = cv2.Canny(template_blur, threshold1=100, threshold2=200)
        
        original_w, original_h = template.shape[::-1]

        # Iterate over scales to resize templates
        for scale in scales:
            scaled_w = int(original_w * scale)
            scaled_h = int(original_h * scale)
            if scaled_w <= 0 or scaled_h <= 0:
                continue  # Skip invalid sizes

            template_resized = cv2.resize(template_edges, (scaled_w, scaled_h))

            # Prepare a list to hold all rotated templates
            templates = []

            # Resize and rotate the template
            for i in range(4):
                if i == 0:
                    template_rotated = template_resized.copy()
                else:
                    # Rotate 90 degrees clockwise for each iteration
                    template_rotated = cv2.rotate(template_rotated, cv2.ROTATE_90_CLOCKWISE)
                templates.append(template_rotated)

            # Perform template matching for each rotated template
            for template_rotated in templates:
                w_rotated, h_rotated = template_rotated.shape[::-1]

                # Perform template matching using CCOEFF_NORMED
                result = cv2.matchTemplate(img_edges, template_rotated, cv2.TM_CCOEFF_NORMED)

                # Find locations exceeding the threshold
                loc = np.where(result >= threshold)

                # Iterate over all matched locations
                for pt in zip(*loc[::-1]):  # Switch columns and rows
                    x1, y1 = pt
                    x2, y2 = x1 + w_rotated, y1 + h_rotated
                    box = [x1, y1, x2, y2]

                    # Check for overlap with existing detections using IoU
                    overlap = False
                    for existing_box in detected_points:
                        xa1, ya1, xa2, ya2 = existing_box
                        inter_x1 = max(x1, xa1)
                        inter_y1 = max(y1, ya1)
                        inter_x2 = min(x2, xa2)
                        inter_y2 = min(y2, ya2)
                        inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
                        box_area = (x2 - x1) * (y2 - y1)
                        existing_area = (xa2 - xa1) * (ya2 - ya1)
                        union_area = box_area + existing_area - inter_area

                        if union_area == 0:
                            continue
                        iou = inter_area / union_area
                        if iou > 0.3:  # IoU threshold for overlap suppression
                            overlap = True
                            break

                    if not overlap:
                        # Draw rectangle on the detected image
                        cv2.rectangle(img_detected, (x1, y1), (x2, y2), (0, 0, 255), 2)
                        detected_points.append([x1, y1, x2, y2])
                        count_detected += 1

    print(f"Total detections: {count_detected}")

    filename = f"{icon_name}_CNT_{count_detected}.jpg"
    filepath = os.path.join(output_folder, filename)
    print(filepath)

    # Optionally, save the detected image
    cv2.imwrite(filepath, img_detected)

In [3]:
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Presentation/test'
os.makedirs(output_folder, exist_ok=True)
print(f'output_folder: {output_folder}')

output_folder: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Presentation/test


In [4]:
# Load your target image and templates
image_path_1 = 'data/input_file/LEVEL1_POWER_PLAN_page_0001.jpg'
image_path_2 = 'data/input_file/LEVEL_1_POWER_PLAN_page_0002.jpg'
image_path_3 = 'data/input_file/LEVEL_1_POWER_PLAN_page_0003.jpg'

# template_paths = ['data/test.jpg', 'data/test1.jpg', 'data/test1.jpg', 'data/test2.jpg', 'data/test3.jpg', 'data/test4.jpg', 'data/test5.jpg', 'data/test6.jpg']
template_paths = ['data/equipment_file/wall_mounted_duplex_receptecle.jpg']


# Read the images
templates = [cv2.imread(template_path) for template_path in template_paths]

In [5]:
# Define scales and angles
scales = np.arange(0.4, 1.4, 0.1)

# Define threshold
threshold = 0.6  # Adjust based on experimentation

icon_name = 'Page_1_test'

# Call the detection function
detect_icons(image_path_1, template_paths, scales, threshold, output_folder, icon_name)

Total detections: 126
C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Presentation/test\Page_1_test_CNT_126.jpg


In [ ]:
# def rotate_image(image, angle):
#     (h, w) = image.shape[:2]
#     center = (w / 2, h / 2)
#     M = cv2.getRotationMatrix2D(center, angle, 1.0)
#     cos = np.abs(M[0, 0])
#     sin = np.abs(M[0, 1])
#     new_w = int((h * sin) + (w * cos))
#     new_h = int((h * cos) + (w * sin))
#     M[0, 2] += (new_w / 2) - center[0]
#     M[1, 2] += (new_h / 2) - center[1]
#     rotated = cv2.warpAffine(image, M, (new_w, new_h))
#     return rotated

# def multi_scale_and_angle_template_matching(image, templates, threshold=0.7):
#     bounding_boxes = []

#     for template in templates:
#         h, w = template.shape[:2]
        
#         # Loop over scales
#         for scale in np.linspace(0.5, 1.5, num=10):  # Adjust scale range as needed
#             resized_template = cv2.resize(template, (int(w * scale), int(h * scale)))
#             res_h, res_w = resized_template.shape[:2]

#             # Skip if resized template is larger than the image
#             if res_h > image.shape[0] or res_w > image.shape[1]:
#                 continue

#             # Loop over angles
#             for angle in [90, 180, 270]:  # Adjust angle step as needed
#                 M = cv2.getRotationMatrix2D((res_w / 2, res_h / 2), angle, 1.0)
#                 rotated_template = cv2.warpAffine(resized_template, M, (res_w, res_h))

#                 # Template matching
#                 result = cv2.matchTemplate(image, rotated_template, cv2.TM_CCOEFF_NORMED)
#                 loc = np.where(result >= threshold)

#                 # Record all locations where matches are found
#                 for pt in zip(*loc[::-1]):  # Switch x and y coordinates
#                     bounding_boxes.append((pt[0], pt[1], res_w, res_h))  # (x, y, width, height)

#     return bounding_boxes

# def non_max_suppression(boxes, overlapThresh=0.7):
#     if len(boxes) == 0:
#         return []

#     boxes = np.array(boxes)
#     pick = []

#     x1 = boxes[:, 0]
#     y1 = boxes[:, 1]
#     x2 = boxes[:, 0] + boxes[:, 2]
#     y2 = boxes[:, 1] + boxes[:, 3]

#     area = (x2 - x1) * (y2 - y1)
#     idxs = np.argsort(y2)

#     while len(idxs) > 0:
#         last = len(idxs) - 1
#         i = idxs[last]
#         pick.append(i)

#         xx1 = np.maximum(x1[i], x1[idxs[:last]])
#         yy1 = np.maximum(y1[i], y1[idxs[:last]])
#         xx2 = np.minimum(x2[i], x2[idxs[:last]])
#         yy2 = np.minimum(y2[i], y2[idxs[:last]])

#         w = np.maximum(0, xx2 - xx1)
#         h = np.maximum(0, yy2 - yy1)
#         overlap = (w * h) / area[idxs[:last]]

#         idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

#     return boxes[pick].astype("int")

In [ ]:
# # Perform multi-scale and multi-angle template matching
# bounding_boxes = multi_scale_and_angle_template_matching(image, templates)

# # Apply non-maxima suppression to reduce overlapping boxes
# filtered_boxes = non_max_suppression(bounding_boxes)

# # Draw bounding boxes on the original image for visualization
# for (x, y, w, h) in filtered_boxes:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


# # Create a filename based on scale and threshold
# output_folder = 'output'  # Specify your output folder
# os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
# filename = f"detected_scale_1.jpg"
# filepath = os.path.join(output_folder, filename)

# # Save the detected image
# cv2.imwrite(filepath, image)

# # Print the count of detected items
# print(f'File Stored at: {filepath}\nfile_name: {filename}')

# # Show the result
# cv2.imshow("Detected Icons", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()